# Scikit-Learn 내장 데이터셋: Optical Recognition of Handwritten Digits Dataset

이 노트북은 scikit-learn에 내장된 `load_digits` 함수를 사용하여 손글씨 숫자 데이터셋을 분석하고 분류하는 과정을 다룹니다.

### 데이터셋 정보
- **데이터 구성**: 1,797개의 8x8 픽셀 이미지 데이터
- **클래스**: 0부터 9까지의 숫자 (총 10개 클래스)
- **특성(Features)**: 각 픽셀의 밝기를 나타내는 64개(8x8)의 정수 값 (0~16 범위)
- **목표**: 8x8 이미지를 입력받아 해당 이미지가 어떤 숫자인지 예측

이 데이터셋은 MNIST 데이터셋의 축소판 버전으로, 하드웨어 성능의 제약 없이 머신러닝 알고리즘을 테스트하기에 적합합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

## 1. 데이터 로드 및 탐색적 데이터 분석 (EDA)

In [ ]:
# 데이터 로드
digits = load_digits()
X = digits.data
y = digits.target

print(f"데이터 크기: {X.shape}")
print(f"타겟 클래스: {np.unique(y)}")

In [ ]:
# 샘플 데이터 시각화
plt.figure(figsize=(10, 4))
for index, (image, label) in enumerate(zip(digits.images[:10], digits.target[:10])):
    plt.subplot(2, 5, index + 1)
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title(f'Label: {label}')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# 클래스 분포 확인
plt.figure(figsize=(8, 5))
sns.countplot(x=y)
plt.title('Target Class Distribution')
plt.xlabel('Digit Class')
plt.ylabel('Count')
plt.show()

## 2. 데이터 전처리 및 특성 엔지니어링

In [ ]:
# 데이터 분할 (학습용 80%, 테스트용 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

## 3. 모델링 (6가지 모델 평가)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier

# 베이스라인 모델 성능 측정을 위한 딕셔너리
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVC': SVC(),
    'Random Forest': RandomForestClassifier(random_state=42),
    'KNN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Extra Trees': ExtraTreesClassifier(random_state=42)
}

model_results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    model_results[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")

# 성능 기준 정렬 후 상위 4개 모델 선정
sorted_models = sorted(model_results.items(), key=lambda x: x[1], reverse=True)
top_4_model_names = [m[0] for m in sorted_models[:4]]
print(f"\n상위 4개 모델: {top_4_model_names}")

## 4. 하이퍼파라메터 튜닝 및 앙상블 모델

In [ ]:
from sklearn.ensemble import VotingClassifier

# 튜닝할 파라미터 정의 (예시)
param_grids = {
    'SVC': {'C': [0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1], 'probability': [True]},
    'Random Forest': {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]},
    'KNN': {'n_neighbors': [3, 5, 7]},
    'Extra Trees': {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]},
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'Gradient Boosting': {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1]}
}

best_estimators = []

for name in top_4_model_names:
    print(f"{name} 튜닝 중...")
    grid = GridSearchCV(models[name], param_grids[name], cv=3, n_jobs=-1)
    grid.fit(X_train_scaled, y_train)
    best_estimators.append((name, grid.best_estimator_))
    print(f"{name} 최적 파라미터: {grid.best_params_}")

# 소프트 보팅 앙상블 생성
ensemble_model = VotingClassifier(estimators=best_estimators, voting='soft')
ensemble_model.fit(X_train_scaled, y_train)

print("\n앙상블 모델 학습 완료")

## 5. 최종 모델 평가

In [ ]:
y_pred_final = ensemble_model.predict(X_test_scaled)

print("### 앙상블 모델 최종 성능 ###")
print(f"최종 정확도 (Accuracy): {accuracy_score(y_test, y_pred_final):.4f}")
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred_final))

# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, y_pred_final)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Final Ensemble Model Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()